In [1]:
# Run this notebook to explore the results of baseline model
import pandas as pd 
import numpy as np 

data = pd.read_csv('../data/clean_data/baseline_input.csv')
data.head()

data = data.fillna(value=' ', axis=1)

from sklearn.model_selection import train_test_split
X = data[['BENZI_man', 'citation_man', 'initial_man', 'final_man']]
Y = data[['BENZI_IN_SOURCE_teo', 'citation_teo', 'initial_teo', 'final_teo']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [2]:
# predict the majority teochew from each mandarin tone/initial/final 
pred = [] 
X_test = X_test.reset_index()
Y_test = Y_test.reset_index()
for i in range(len(X_test)):
    most_tone = Y_train[X_train['citation_man'] == X_test.iloc[i]['citation_man']].citation_teo.mode()[0]
    #most_tone =  " " if most_tone.empty else most_tone[0]
    most_initial = Y_train[X_train['initial_man'] == X_test.iloc[i]['initial_man']].initial_teo.mode()[0]
    #most_initial = " " if most_initial.empty else most_initial[0]
    most_final = Y_train[X_train['final_man'] == X_test.iloc[i]['final_man']].final_teo.mode()[0]
#     most_final = " " if most_final.empty else most_final[0]
#     if most_final == " ":
#         print(most_final)
    pred.append([most_tone, most_initial, most_final])
pred_pd = pd.DataFrame(pred, columns=['pred_tone', 'pred_initial', 'pred_final'])

In [3]:
merged_pd = pd.concat([X_test, Y_test, pred_pd], axis=1)

merged_pd.describe()

,index,citation_man,index,citation_teo,pred_tone
count,161.000000,161.000000,161.000000,161.000000,161.000000
mean,408.962733,2.627329,408.962733,66.677019,104.577640
std,237.734303,1.182264,237.734303,69.193262,79.854057
min,2.000000,1.000000,2.000000,4.000000,33.000000
25%,210.000000,2.000000,210.000000,33.000000,53.000000
50%,398.000000,3.000000,398.000000,53.000000,55.000000
75%,611.000000,4.000000,611.000000,55.000000,213.000000
max,799.000000,4.000000,799.000000,213.000000,213.000000


### Calculates the accuracy for tone, initial, and final 

In [4]:
tone_acc = len(merged_pd[merged_pd.citation_teo==merged_pd.pred_tone])
initial_acc = len(merged_pd[merged_pd.initial_teo==merged_pd.pred_initial])
final_acc = len(merged_pd[merged_pd.final_teo==merged_pd.pred_final])
tone_acc, initial_acc, final_acc

(91, 109, 51)

In [5]:
100*tone_acc/161, 100*initial_acc/161, 100*final_acc/161

(56.52173913043478, 67.70186335403727, 31.677018633540374)

In [6]:
reduced_df = merged_pd[['BENZI_IN_SOURCE_teo', 'citation_man', 'initial_man', 'final_man', \
                        'citation_teo', 'initial_teo', 'final_teo', 'pred_tone', \
                        'pred_initial', 'pred_final']]
#reduced_df.to_csv('../output/baseline.csv', index = None, header=True)
reduced_df.head()

,BENZI_IN_SOURCE_teo,citation_man,initial_man,final_man,citation_teo,initial_teo,final_teo,pred_tone,pred_initial,pred_final
0,餡,4,x,ian,11,,ã∼,213,s,iəm
1,蜘,1,zh,i,33,t,i,33,ts,i
2,瘌,4,l,a,4,l,aʔ,213,l,aʔ
3,隔,2,g,e,21,k,eʔ,55,k,o
4,殼,2,k,e,4,kʰ,ak,55,kʰ,o


In [7]:
reduced_df[(reduced_df.citation_teo==reduced_df.pred_tone)&
          (reduced_df.initial_teo==reduced_df.pred_initial)&
          (reduced_df.final_teo==reduced_df.pred_final)].drop_duplicates('initial_man')

,BENZI_IN_SOURCE_teo,citation_man,initial_man,final_man,citation_teo,initial_teo,final_teo,pred_tone,pred_initial,pred_final
14,臺,2,t,ai,55,tʰ,ai,55,tʰ,ai
16,走,3,z,ou,53,ts,au,53,ts,au
17,好,3,h,ao,53,h,o,53,h,o
36,樓,2,l,ou,55,l,au,55,l,au
51,光,1,g,uang,33,k,ɯŋ,33,k,ɯŋ
71,褲,4,k,u,213,kʰ,ou,213,kʰ,ou
92,莊,1,zh,uang,33,ts,ɯŋ,33,ts,ɯŋ
101,隨,2,s,ui,55,s,ui,55,s,ui
124,配,4,p,ei,213,pʰ,ue,213,pʰ,ue
145,晏,4,y,an,213,,ũã∼,213,,ũã∼
